In [ ]:
from PIL import Image

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

Image.MAX_IMAGE_PIXELS = None

# Colormap

In [ ]:
min_latitude = -60
max_latitude = 70
desiredSizes = (16384, 8192)

colormap = Image.open('inputs/bluemarble/NASAaugust.png')

# Get the width and height of the image
width, height = colormap.size

# Calculate the y-coordinates for the clipping range
min_y = int((90 - max_latitude) / 180 * height)
max_y = int((90 - min_latitude) / 180 * height)

# Create a new image with the clipped region
colormap = colormap.crop((0, min_y, width, max_y)).resize(desiredSizes, Image.Resampling.LANCZOS)
colormap.save('outputs/colormap.png')

# Loading

In [ ]:
## Load name database
file_location = 'inputs/databases/worldcities.csv'
worldcities = pd.read_csv(file_location, delimiter=';')
print(worldcities.head())

In [ ]:
## Load CSVs
definitions = pd.read_csv('inputs/definition.csv', delimiter=';')
definitions = definitions[['R','G','B']].to_numpy()
definitions

In [ ]:
locators = {}
with open('inputs/city_locators.txt', 'r') as file:
    contents = file.read().splitlines()
    for line in range(8, len(contents)-4, 6):
        ids = int(contents[line].split("=")[1])
        
        temp = contents[line+1].split(" ")[1:-1]

        newY = (6144-int(temp[2]))/6144*8192
        locators[ids] = (int(temp[0]), newY)

In [ ]:
names = {}
with open(f'inputs/provincenames_l_english.yml', 'r') as file:
    contents = file.read().splitlines()
    for line in range(1, len(contents)):
        names[line] = contents[line].split('"')[1]

In [ ]:
imgProvs = Image.open('inputs/provinces.png')

In [ ]:
temp = np.array(worldcities['lng'])
worldcities['x'] = (0.5+temp/360)*16384

temp = np.array(worldcities['lat'])
worldcities['y'] = (70-temp)*(8192/130)
worldcities.head(2)

# Coordinates

In [ ]:
latlong = {}
for i,ids in enumerate(locators):
    x, y = locators[ids]

    long = 360*(x/16384-0.5)
    lat = 70-130/8192*y
    
    latlong[ids] = (lat,long)

# Names

In [ ]:
def squaredDistance(pointA, pointB):
    dist, m = 0, len(pointA)
    for dim in range(m):
        dist += (pointA[dim]-pointB[dim])**2
    return dist

def nearestCity(xy: tuple, locators: dict):
    distances = {}
    for ids in locators:
        distances[ids] = squaredDistance(xy, locators[ids])

    nearest = min(distances, key=distances.get)
    return nearest

In [ ]:
used = []
for i in tqdm(range(len(worldcities))):
    city = worldcities.iloc[i,:]
    nearest = nearestCity((city['x'],city['y']), locators)

    if nearest not in used:
        names[nearest] = city['city_ascii']
        used.append(nearest)

# Areas

In [ ]:
pixels, counts = np.unique(np.array(imgProvs).reshape(-1,3), axis=0, return_counts=True)

In [ ]:
def area_coef(lat, long, deltaLat, deltaLong): #Area in sq miles
    earthR = 6378.137

    d1 = np.sin(0.5*deltaLat)**2
    d1 = 2*earthR*np.arctan2(np.sqrt(d1),np.sqrt(1-d1))

    d2 = np.cos(lat)**2*np.sin(0.5*deltaLong)**2
    d2 = 2*earthR*np.arctan2(np.sqrt(d2),np.sqrt(1-d2))

    area = d1*d2 #Area in km2
    #area *= 0.3861
    return area

inhabitables = list(range(4883,5692+1))
inhabitables.append(1)
areas = {}
diagonals = {}
for j, ids in tqdm(enumerate(locators)):
    if ids in inhabitables: continue

    provColor = tuple(definitions[ids])
    for i, color in enumerate(pixels):
        if provColor==tuple(color):
            break
    count = counts[i]

    x, y = locators[ids]
    deltaLong = np.deg2rad(abs(360*((x+1)/16384-0.5)-360*(x/16384-0.5)))
    deltaLat = np.deg2rad(abs((70-130/8192*(y+1))-(70-130/8192*y)))

    lat = np.deg2rad(latlong[ids][0])
    long = np.deg2rad(latlong[ids][1])

    areas[ids] = round(count*area_coef(lat, long, deltaLat, deltaLong), 3)
    diagonals[ids] = np.sqrt(areas[ids])

# Tree/Rock layers
With advanced real data (temperature, rainfall, accurate conversion to lat long) we can have a better model to paint tree / rock / grasses layers

# Saving province data

In [ ]:
# Province parameters
climateKeys = {'Arid':-1, 'Warm':0, 'Mild':1, 'Normal':2, 'Severe':3}
with open('outputs/provSetup.txt','w') as file:
    file.write('namespace = provSetup\n\nprovSetup.1 = {\n\ttype = country_event\n\n\thidden = yes\n\n\timmediate = {\n')
    for ids in names:
        if ids in inhabitables: continue
        file.write(f'\t\tp:{ids} = {"{"} ')
        #XY
        file.write(f'set_variable = {"{"} name = dat_x value = {locators[ids][0]} {"}"} ')
        file.write(f'set_variable = {"{"} name = dat_y value = {6144-locators[ids][1]} {"}"} ')
        #latitude-longitude
        file.write(f'set_variable = {"{"} name = dat_lat value = {round(latlong[ids][0],2)} {"}"} ')
        file.write(f'set_variable = {"{"} name = dat_lng value = {round(latlong[ids][1],2)} {"}"} ')
        #Area in square miles
        file.write(f'set_variable = {"{"} name = dat_area value = {areas[ids]} {"}"} ')
        #Square root of the area
        file.write(f'set_variable = {"{"} name = dat_dist value = {round(diagonals[ids],1)} {"}"} ')
        #Rain in inches
        file.write(f'set_variable = {"{"} name = dat_rain value = {round(rainfall[ids],2)} {"}"} ')
        file.write(f'set_variable = {"{"} name = rainlocal value = {round(rainfall[ids],2)} {"}"} ')
        #Temperature in degrees celsius
        file.write(f'set_variable = {"{"} name = dat_temp value = {temperature[ids]} {"}"} ')
        file.write(f'set_variable = {"{"} name = templocal value = {temperature[ids]} {"}"} ')
        #Elevation in feet above sea level
        file.write(f'set_variable = {"{"} name = dat_elev value = {elevation[ids]} {"}"} ')
        #Winter harshness
        for id1 in climate:
            for id2 in climate[id1]:
                if id2==ids:
                    file.write(f'set_variable = {"{"} name = dat_type value = flag:{id1} {"}"} ') 
                    break
        file.write('}\n')
    file.write('\t}\n\n}')

In [ ]:
### Write localization. This is not useful unless you want to overwrite manual changes to localization
with open('outputs/provincenames_l_english.yml', 'w') as file:
    file.write('l_english:\n')
    for ids in names:
        file.write(f' PROV{ids}:0 "{names[ids]}"\n')

In [ ]:
print("\n".join(["2",
 'Elliot Lake',
 '23771.687 sq.miles',
 '154.18069593823995 miles',
 "(4407, 1260)",
 "(46.37081550517141, -83.16650390625)",
 '30.82677165342 inches of rain',
 '2.7 degrees celcius',
 '204 frost days',
 '541ft above sea level']))